박수빈

In [13]:
from tqdm.notebook import tqdm

In [1]:
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import ElectraTokenizer, ElectraModel, BertTokenizer, BertModel

2023-01-16 20:47:01.883950: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 20:47:02.542507: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-16 20:47:02.542595: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-16 20:47:02.542601: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
torch.autograd.set_detect_anomaly(True)

In [15]:
from scipy.spatial.distance import cosine

## ELECTRA PRACTICE

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [17]:
electra_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
electra_model = ElectraModel.from_pretrained("monologg/koelectra-small-v3-discriminator")

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
bert_tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
bert_model = BertModel.from_pretrained('monologg/kobert')

In [19]:
text1 = "한국어 ELECTRA를 공유합니다."
text2 = '한국의 국어로 된 ELECTRA를 공유합니다.'

In [20]:
electra_tokenizer.tokenize(text1), electra_tokenizer.tokenize(text2)

(['한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.'],
 ['한국',
  '##의',
  '국어',
  '##로',
  '된',
  'EL',
  '##EC',
  '##TRA',
  '##를',
  '공유',
  '##합니다',
  '.'])

In [21]:
electra_tensor1 = torch.tensor(electra_tokenizer.encode(text1)).unsqueeze(0)
electra_tensor2 = torch.tensor(electra_tokenizer.encode(text2)).unsqueeze(0)
bert_tensor1 = torch.tensor(bert_tokenizer.encode(text1)).unsqueeze(0)
bert_tensor2 = torch.tensor(bert_tokenizer.encode(text2)).unsqueeze(0)

In [22]:
with torch.no_grad():
    electra_embedded_tensor1 = torch.Tensor.numpy(electra_model(electra_tensor1)[0][0][0])
    electra_embedded_tensor2 = torch.Tensor.numpy(electra_model(electra_tensor2)[0][0][0])
    bert_embedded_tensor1 = torch.Tensor.numpy(bert_model(bert_tensor1)[0][0][0])
    bert_embedded_tensor2 = torch.Tensor.numpy(bert_model(bert_tensor2)[0][0][0])
    electra_distance = cosine(electra_embedded_tensor1, electra_embedded_tensor2)
    bert_distance = cosine(bert_embedded_tensor1, bert_embedded_tensor2)

    print(
        electra_distance,
        bert_distance,
        f'difference: {electra_distance - bert_distance}',
        sep='\n'
    )

0.00993812084197998
0.040048182010650635
difference: -0.030110061168670654


## PyTorch Syntax

In [7]:
model = nn.Linear(in_features=1, out_features=1)

x_data = torch.Tensor([1, 2, 3, 4, 5]).view(-1, 1)
y_data = torch.Tensor([2, 4, 6, 8, 10]).view(-1, 1)

criterion = nn.MSELoss(model.parameters())
optimizer = torch.optim.Adam()

for _ in range(100):
    pred = model(x_data)
    loss = criterion(pred, y_data)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

[Parameter containing:
tensor([[-0.8512]], requires_grad=True), Parameter containing:
tensor([-0.5640], requires_grad=True)]
